<a href="https://colab.research.google.com/github/felizzi/PLS_Simpy/blob/main/Simpy_tutorial_Patient_Level_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the SimPy library

In [5]:
!pip install simpy

# Initialization
Initialization of the *Patient* class and definition of
the *visit_hospital* function inside the class

In [6]:
import simpy
import random

class Patient:
    def __init__(self, name, env, resource):
        self.name = name
        self.env = env
        self.resource = resource
        self.action = env.process(self.visit_hospital())

    def visit_hospital(self):
        print(f"{self.name} arrives at the hospital at {self.env.now}")

        # Request a bed in the hospital
        with self.resource.request() as req:
            yield req
            print(f"{self.name} gets a bed at {self.env.now}")

            # The amount of time the patient spends in the bed
            treatment_time = random.randint(5, 20)
            yield self.env.timeout(treatment_time)

            print(f"{self.name} leaves the hospital at {self.env.now}")

# Initialize the SimPy environment
env = simpy.Environment()

# Create a resource with 3 beds
beds = simpy.Resource(env, capacity=3)

# Generate 5 patients
for i in range(5):
    patient_name = f"Patient-{i}"
    Patient(patient_name, env, beds)

# Run the simulation
env.run()

Patient-0 arrives at the hospital at 0
Patient-1 arrives at the hospital at 0
Patient-2 arrives at the hospital at 0
Patient-3 arrives at the hospital at 0
Patient-4 arrives at the hospital at 0
Patient-0 gets a bed at 0
Patient-1 gets a bed at 0
Patient-2 gets a bed at 0
Patient-1 leaves the hospital at 10
Patient-3 gets a bed at 10
Patient-0 leaves the hospital at 16
Patient-4 gets a bed at 16
Patient-3 leaves the hospital at 17
Patient-2 leaves the hospital at 19
Patient-4 leaves the hospital at 29


Simple dynamic allocation of resources, as doctors are added when the waiting time exceeds a given threshold

In [7]:
import simpy
import random

# Initialize variables to hold total waiting times and number of patients
total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Threshold for average waiting time (in arbitrary time units)

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits == 0:
            continue
        avg_wait_time = total_doctor_wait / num_doctor_visits
        print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
        if avg_wait_time > waiting_time_threshold:
            new_capacity = doctor.capacity + 1
            doctor = simpy.Resource(env, capacity=new_capacity)
            print(f"Added an extra doctor at {env.now}, new capacity: {new_capacity}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor resource with initial capacity of 1
doctor = simpy.Resource(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Generate patients
for i in range(20):
    env.process(patient(env, f"Patient-{i}", doctor))
    next_patient_in = random.randint(1, 4)
    #yield env.timeout(next_patient_in)

# Run the simulation
env.run(until=100)


Patient-0 starts consultation at 0, waited 0
Patient-0 finishes consultation at 7
Patient-1 starts consultation at 7, waited 7
Average doctor waiting time at 10 is 3.5
Patient-1 finishes consultation at 15
Patient-2 starts consultation at 15, waited 15
Average doctor waiting time at 20 is 7.333333333333333
Patient-2 finishes consultation at 20
Patient-3 starts consultation at 20, waited 20
Patient-3 finishes consultation at 28
Patient-4 starts consultation at 28, waited 28
Average doctor waiting time at 30 is 14.0
Added an extra doctor at 30, new capacity: 2
Patient-4 finishes consultation at 36
Patient-5 starts consultation at 36, waited 36
Average doctor waiting time at 40 is 17.666666666666668
Added an extra doctor at 40, new capacity: 3
Patient-5 finishes consultation at 46
Patient-6 starts consultation at 46, waited 46
Average doctor waiting time at 50 is 21.714285714285715
Added an extra doctor at 50, new capacity: 4
Patient-6 finishes consultation at 53
Patient-7 starts consulta

Overview of a slightly more complex process, with waiting times at:

*   Registration
*   Doctor visit
*   Pharmacy


In [5]:
import simpy
import random

# Initialize variables to hold total waiting times
total_registration_wait = 0
total_doctor_wait = 0
total_pharmacy_wait = 0
total_patients = 5

def patient(env, name, registration, doctor, pharmacy):
    global total_registration_wait, total_doctor_wait, total_pharmacy_wait

    print(f"{name} enters the hospital at {env.now}")

    # Step 1: Registration
    arrival_time = env.now
    with registration.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_registration_wait += wait_time
        print(f"{name} starts registration at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(1, 3))
        print(f"{name} finishes registration at {env.now}")

    # Step 2: Doctor Consultation
    arrival_time = env.now
    with doctor.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        print(f"{name} starts consultation at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

    # Step 3: Pharmacy
    arrival_time = env.now
    with pharmacy.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_pharmacy_wait += wait_time
        print(f"{name} starts medication pickup at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(2, 5))
        print(f"{name} finishes medication pickup and leaves the hospital at {env.now}")

# Initialize SimPy environment
env = simpy.Environment()

# Initialize resources: registration desk, doctor, and pharmacy
registration = simpy.Resource(env, capacity=1)
doctor = simpy.Resource(env, capacity=1)
pharmacy = simpy.Resource(env, capacity=1)

# Generate 5 patients
for i in range(total_patients):
    env.process(patient(env, f"Patient-{i}", registration, doctor, pharmacy))

# Run the simulation
env.run()

# Calculate average waiting time for each resource
print("\nAverage waiting times:")
print(f"Registration: {total_registration_wait / total_patients:.2f}")
print(f"Doctor: {total_doctor_wait / total_patients:.2f}")
print(f"Pharmacy: {total_pharmacy_wait / total_patients:.2f}")


Patient-0 enters the hospital at 0
Patient-1 enters the hospital at 0
Patient-2 enters the hospital at 0
Patient-3 enters the hospital at 0
Patient-4 enters the hospital at 0
Patient-0 starts registration at 0, waited 0
Patient-0 finishes registration at 2
Patient-0 starts consultation at 2, waited 0
Patient-1 starts registration at 2, waited 2
Patient-1 finishes registration at 5
Patient-2 starts registration at 5, waited 5
Patient-2 finishes registration at 6
Patient-3 starts registration at 6, waited 6
Patient-3 finishes registration at 8
Patient-4 starts registration at 8, waited 8
Patient-4 finishes registration at 9
Patient-0 finishes consultation at 12
Patient-0 starts medication pickup at 12, waited 0
Patient-1 starts consultation at 12, waited 7
Patient-0 finishes medication pickup and leaves the hospital at 16
Patient-1 finishes consultation at 20
Patient-1 starts medication pickup at 20, waited 0
Patient-2 starts consultation at 20, waited 14
Patient-1 finishes medication pi

Additional way to increase capacity dynamically, without re-initializing the class

In [ ]:
import simpy
import random

total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Time units

class Doctor(simpy.Resource):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)

    def add_doctor(self):
        self._capacity += 1

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits > 0:
            avg_wait_time = total_doctor_wait / num_doctor_visits
            print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
            if avg_wait_time > waiting_time_threshold:
                doctor.add_doctor()
                print(f"Added an extra doctor at {env.now}, new capacity: {doctor.capacity}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")



def generate_patients(env, doctor):
    for i in range(20):
        env.process(patient(env, f"Patient-{i}", doctor))
        next_patient_in = random.randint(1, 4)
        yield env.timeout(next_patient_in)

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor resource with initial capacity of 1
doctor = Doctor(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Start the patient generation process
env.process(generate_patients(env, doctor))

# Run the simulation until a specified time
env.run(until=100)


Patient-0 starts consultation at 0, waited 0
Patient-0 finishes consultation at 8
Patient-1 starts consultation at 8, waited 7
Average doctor waiting time at 10 is 3.5
Patient-1 finishes consultation at 18
Patient-2 starts consultation at 18, waited 15
Average doctor waiting time at 20 is 7.333333333333333
Patient-2 finishes consultation at 25
Patient-3 starts consultation at 25, waited 20
Average doctor waiting time at 30 is 10.5
Added an extra doctor at 30, new capacity: 2
Patient-4 starts consultation at 30, waited 23
Patient-3 finishes consultation at 34
Patient-5 starts consultation at 34, waited 26
Patient-4 finishes consultation at 38
Patient-6 starts consultation at 38, waited 26
Average doctor waiting time at 40 is 16.714285714285715
Added an extra doctor at 40, new capacity: 3
Patient-7 starts consultation at 41, waited 27
Patient-5 finishes consultation at 43
Patient-8 starts consultation at 43, waited 25
Patient-6 finishes consultation at 48
Patient-9 starts consultation at

Example of use of inheritance, in which both a generalist and a specialist doctor are defined

In [ ]:
import simpy
import random

total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Time units

class Doctor(simpy.Resource):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "General"

    def add_doctor(self):
        self._capacity += 1

class SpecialistDoctor(Doctor):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "Specialist"

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits > 0:
            avg_wait_time = total_doctor_wait / num_doctor_visits
            print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
            if avg_wait_time > waiting_time_threshold:
                doctor.add_doctor()
                if doctor.doctor_type == "General":
                    doctor.__class__ = SpecialistDoctor
                print(f"Added an extra {doctor.doctor_type} doctor at {env.now}, new capacity: {doctor.capacity}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation with {doctor.doctor_type} at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

def generate_patients(env, doctor):
    for i in range(20):
        env.process(patient(env, f"Patient-{i}", doctor))
        next_patient_in = random.randint(1, 4)
        yield env.timeout(next_patient_in)

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor resource with initial capacity of 1
doctor = Doctor(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Start the patient generation process
env.process(generate_patients(env, doctor))

# Run the simulation until a specified time
env.run(until=100)


Patient-0 starts consultation with General at 0, waited 0
Patient-0 finishes consultation at 6
Patient-1 starts consultation with General at 6, waited 4
Average doctor waiting time at 10 is 2.0
Patient-1 finishes consultation at 11
Patient-2 starts consultation with General at 11, waited 8
Patient-2 finishes consultation at 17
Patient-3 starts consultation with General at 17, waited 10
Average doctor waiting time at 20 is 5.5
Patient-3 finishes consultation at 27
Patient-4 starts consultation with General at 27, waited 17
Average doctor waiting time at 30 is 7.8
Patient-4 finishes consultation at 35
Patient-5 starts consultation with General at 35, waited 22
Average doctor waiting time at 40 is 10.166666666666666
Added an extra General doctor at 40, new capacity: 2
Patient-6 starts consultation with General at 40, waited 25
Patient-5 finishes consultation at 45
Patient-7 starts consultation with General at 45, waited 29
Patient-6 finishes consultation at 47
Patient-8 starts consultatio

Simulation with a generalist and a specialist doctor


In [ ]:
# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor resource with initial capacity of 1
doctor = Doctor(env, capacity=2)

# Initialize specialist doctor resource with initial capacity of 0
specialist_doctor = SpecialistDoctor(env, capacity=1)

def monitor(env, doctor, specialist_doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits > 0:
            avg_wait_time = total_doctor_wait / num_doctor_visits
            print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
            if avg_wait_time > waiting_time_threshold:
                specialist_doctor.add_doctor()
                print(f"Added an extra {specialist_doctor.doctor_type} doctor at {env.now}, new capacity: {specialist_doctor.capacity}")

def generate_patients(env, doctor, specialist_doctor):
    for i in range(50):  # Increased patient count
        if specialist_doctor.capacity > 0 and random.random() < 0.5:
            env.process(patient(env, f"Patient-{i}", specialist_doctor))
        else:
            env.process(patient(env, f"Patient-{i}", doctor))

        next_patient_in = random.randint(1, 4)
        yield env.timeout(next_patient_in)

# Start the monitoring process
env.process(monitor(env, doctor, specialist_doctor))

# Start the patient generation process
env.process(generate_patients(env, doctor, specialist_doctor))

# Run the simulation until a specified time
env.run(until=200)  # Increased simulation time


Patient-0 starts consultation with General at 0, waited 0
Patient-1 starts consultation with General at 3, waited 0
Patient-0 finishes consultation at 5
Patient-2 starts consultation with Specialist at 7, waited 0
Average doctor waiting time at 10 is 6.164383561643835
Patient-4 starts consultation with General at 11, waited 0
Patient-1 finishes consultation at 12
Patient-2 finishes consultation at 16
Patient-4 finishes consultation at 16
Patient-3 starts consultation with Specialist at 16, waited 8
Patient-6 starts consultation with General at 18, waited 0
Average doctor waiting time at 20 is 6.026315789473684
Patient-8 starts consultation with General at 21, waited 0
Patient-6 finishes consultation at 23
Patient-9 starts consultation with General at 23, waited 1
Patient-3 finishes consultation at 24
Patient-5 starts consultation with Specialist at 24, waited 9
Patient-5 finishes consultation at 29
Patient-7 starts consultation with Specialist at 29, waited 9
Average doctor waiting tim

Generation of two processes with different priorities and introduction of preemption

In [ ]:
import simpy

def resource_user(env, resource, priority):
    while True:  # Continue trying to use the resource until successful
        try:
            with resource.request(priority=priority) as req:
                yield req  # Wait until the resource is available or the process is preempted
                print(f"Resource being used with priority {priority} at {env.now}")
                yield env.timeout(2)  # Use the resource for 2 time units
                print(f"Resource released with priority {priority} at {env.now}")
                break  # Exit the while loop once the resource use is complete
        except simpy.Interrupt as interrupt:
            # Handle preemption
            print(f"Process with priority {priority} was preempted at {env.now} by {interrupt.cause}")

env = simpy.Environment()
resource = simpy.PreemptiveResource(env, capacity=1)

# Start processes with different priorities
env.process(resource_user(env, resource, priority=1))  # Lower priority process
env.process(resource_user(env, resource, priority=0))  # Higher priority process

# Run the simulation
env.run()



Process with priority 1 was preempted at 0 by <simpy.resources.resource.Preempted object at 0x7cedb8408430>
Resource being used with priority 0 at 0
Resource released with priority 0 at 2
Resource being used with priority 1 at 2
Resource released with priority 1 at 4


In [ ]:
import simpy
import random

total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Time units

class Doctor:
    def __init__(self, env, capacity):
        self.resource = simpy.Resource(env, capacity)
        self.doctor_type = "General"

class SpecialistDoctor(Doctor):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "Specialist"

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits > 0:
            avg_wait_time = total_doctor_wait / num_doctor_visits
            print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
            if avg_wait_time > waiting_time_threshold and doctor.doctor_type == "General":
                doctor.__class__ = SpecialistDoctor
                print(f"Changed doctor type to {doctor.doctor_type} at {env.now}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.resource.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation with {doctor.doctor_type} at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

def generate_patients(env, doctor):
    for i in range(50):
        env.process(patient(env, f"Patient-{i}", doctor))
        next_patient_in = random.randint(1, 4)
        yield env.timeout(next_patient_in)

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor with initial capacity of 1
doctor = Doctor(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Start the patient generation process
env.process(generate_patients(env, doctor))

# Run the simulation until a specified time
env.run(until=100)


Patient-0 starts consultation with General at 0, waited 0
Average doctor waiting time at 10 is 0.0
Patient-0 finishes consultation at 10
Patient-1 starts consultation with General at 10, waited 6
Average doctor waiting time at 20 is 3.0
Patient-1 finishes consultation at 20
Patient-2 starts consultation with General at 20, waited 15
Patient-2 finishes consultation at 28
Patient-3 starts consultation with General at 28, waited 20
Average doctor waiting time at 30 is 10.25
Changed doctor type to General at 30
Patient-3 finishes consultation at 35
Patient-4 starts consultation with General at 35, waited 23
Average doctor waiting time at 40 is 12.8
Changed doctor type to General at 40
Patient-4 finishes consultation at 44
Patient-5 starts consultation with General at 44, waited 28
Patient-5 finishes consultation at 49
Patient-6 starts consultation with General at 49, waited 30
Average doctor waiting time at 50 is 17.428571428571427
Changed doctor type to General at 50
Patient-6 finishes co

In [ ]:
import simpy
import random

total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Time units

class Doctor:
    def __init__(self, env, capacity):
        self.resource = simpy.Resource(env, capacity)
        self.doctor_type = "General"

class SpecialistDoctor(Doctor):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "Specialist"

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits > 0:
            avg_wait_time = total_doctor_wait / num_doctor_visits
            print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
            if avg_wait_time > waiting_time_threshold:
                doctor.__class__ = SpecialistDoctor
                doctor.doctor_type = "Specialist"
                print(f"Changed doctor type to {doctor.doctor_type} at {env.now}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.resource.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation with {doctor.doctor_type} at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

def generate_patients(env, doctor):
    for i in range(50):
        env.process(patient(env, f"Patient-{i}", doctor))
        next_patient_in = random.randint(1, 4)
        yield env.timeout(next_patient_in)

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor with initial capacity of 1
doctor = Doctor(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Start the patient generation process
env.process(generate_patients(env, doctor))

# Run the simulation until a specified time
env.run(until=100)


Patient-0 starts consultation with General at 0, waited 0
Average doctor waiting time at 10 is 0.0
Patient-0 finishes consultation at 10
Patient-1 starts consultation with General at 10, waited 8
Patient-1 finishes consultation at 17
Patient-2 starts consultation with General at 17, waited 14
Average doctor waiting time at 20 is 7.333333333333333
Patient-2 finishes consultation at 22
Patient-3 starts consultation with General at 22, waited 18
Patient-3 finishes consultation at 27
Patient-4 starts consultation with General at 27, waited 20
Average doctor waiting time at 30 is 12.0
Changed doctor type to Specialist at 30
Patient-4 finishes consultation at 37
Patient-5 starts consultation with Specialist at 37, waited 28
Average doctor waiting time at 40 is 14.666666666666666
Changed doctor type to Specialist at 40
Patient-5 finishes consultation at 44
Patient-6 starts consultation with Specialist at 44, waited 34
Average doctor waiting time at 50 is 17.428571428571427
Changed doctor type

In [6]:
import simpy

def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print(f'{name} requesting at {env.now} with priority={prio}')
        yield req
        print(f'{name} got resource at {env.now}')
        yield env.timeout(3)


env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=5))
p2 = env.process(resource_user(2, env, res, wait=0, prio=2))
p3 = env.process(resource_user(3, env, res, wait=2, prio=1))
env.run()


1 requesting at 0 with priority=5
2 requesting at 0 with priority=2
1 got resource at 0
3 requesting at 2 with priority=1
3 got resource at 3
2 got resource at 6


In [8]:

total_doctor_wait = 0
num_doctor_visits = 0
waiting_time_threshold = 8  # Time units

class Doctor(simpy.Resource):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "General"

    def add_doctor(self):
        self._capacity += 1

class SpecialistDoctor(Doctor):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "Specialist"

env = simpy.Environment()
John = SpecialistDoctor(env, capacity = 3)

An example of a Glaucoma clinic.
Individuals are visited by general ophthalmologists first, then referred to glaucoma specialists.
Due to the smaller number of glaucoma specialists, a long wairing time is generated

In [9]:
import numpy as np


class g:
  total_general_ophthalmologist_wait = 0
  total_glaucoma_specialist_wait = 0
  total_patients = 5



def GlaucomaPatient(env, name, ophthalmologist, specialist):
    print(f"{name} enters the system at {env.now}")
    # Step 1: General Ophthalmologist visit
    arrival_time = env.now
    with ophthalmologist.request() as req:
        yield req
        wait_time = env.now - arrival_time
        g.total_general_ophthalmologist_wait += wait_time
        print(f"{name} starts visit at general ophtalmologist at {env.now}, waited {wait_time}")
        yield env.timeout(float(np.random.normal(1, 0.1, 1)[0]))
        print(f"{name} finishes visit at general ophthalmologist at {env.now}")

    # Step 2: Doctor Consultation
    arrival_time = env.now
    with specialist.request() as req:
        yield req
        wait_time = env.now - arrival_time
        g.total_glaucoma_specialist_wait += wait_time
        print(f"{name} starts visit at glaucoma specialist at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes visit at glaucoma specialist at {env.now}")


# Initialize SimPy environment
env = simpy.Environment()

# Initialize resources: general ophthalmologist, glaucoma specialist
general_ophthalmologist = simpy.Resource(env, capacity=2)
glaucoma_specialist = simpy.Resource(env, capacity=1)


# Generate 5 patients
for i in range(g.total_patients):
    env.process(GlaucomaPatient(env, f"Patient-{i}", general_ophthalmologist, glaucoma_specialist))

# Run the simulation
env.run()

# Calculate average waiting time for each resource
print("\nAverage waiting times:")
print(f"General Opthalmologist: {g.total_general_ophthalmologist_wait / g.total_patients:.2f}")
print(f"Glaucoma Specialist: {g.total_glaucoma_specialist_wait / g.total_patients:.2f}")


Patient-0 enters the system at 0
Patient-1 enters the system at 0
Patient-2 enters the system at 0
Patient-3 enters the system at 0
Patient-4 enters the system at 0
Patient-0 starts visit at general ophtalmologist at 0, waited 0
Patient-1 starts visit at general ophtalmologist at 0, waited 0
Patient-0 finishes visit at general ophthalmologist at 0.898896591919035
Patient-0 starts visit at glaucoma specialist at 0.898896591919035, waited 0.0
Patient-2 starts visit at general ophtalmologist at 0.898896591919035, waited 0.898896591919035
Patient-1 finishes visit at general ophthalmologist at 0.9840683136066126
Patient-3 starts visit at general ophtalmologist at 0.9840683136066126, waited 0.9840683136066126
Patient-2 finishes visit at general ophthalmologist at 1.9291641652667433
Patient-4 starts visit at general ophtalmologist at 1.9291641652667433, waited 1.9291641652667433
Patient-3 finishes visit at general ophthalmologist at 1.9450963697007837
Patient-4 finishes visit at general ophth

In [10]:
import numpy as np

class g:
  total_general_ophthalmologist_wait = 0
  total_glaucoma_specialist_wait = 0
  total_patients = 3



def GlaucomaPatient(env, name, ophthalmologist):
    print(f"{name} enters the system at {env.now}")
    # Step 1: General Ophthalmologist visit
    arrival_time = env.now
    with ophthalmologist.request() as req:
        yield req
        wait_time = env.now - arrival_time
        g.total_general_ophthalmologist_wait += wait_time
        print(f"{name} starts visit at general ophtalmologist at {round(env.now,2)}, waited {round(wait_time,2)}")
        yield env.timeout(float(np.random.normal(1, 0.1, 1)[0]))
        print(f"{name} finishes visit at general ophthalmologist at {round(env.now,2)}")


# Initialize SimPy environment
env = simpy.Environment()

# Initialize resources: general ophthalmologist, glaucoma specialist
general_ophthalmologist = simpy.Resource(env, capacity=2)

# Generate 5 patients
for i in range(g.total_patients):
    env.process(GlaucomaPatient(env, f"Patient-{i}", general_ophthalmologist))

# Run the simulation
env.run()

# Calculate average waiting time for each resource
print("\nAverage waiting times:")
print(f"General Opthalmologist: {g.total_general_ophthalmologist_wait / g.total_patients:.2f}")
print(f"Glaucoma Specialist: {g.total_glaucoma_specialist_wait / g.total_patients:.2f}")


Patient-0 enters the system at 0
Patient-1 enters the system at 0
Patient-2 enters the system at 0
Patient-0 starts visit at general ophtalmologist at 0, waited 0
Patient-1 starts visit at general ophtalmologist at 0, waited 0
Patient-1 finishes visit at general ophthalmologist at 0.87
Patient-2 starts visit at general ophtalmologist at 0.87, waited 0.87
Patient-0 finishes visit at general ophthalmologist at 1.04
Patient-2 finishes visit at general ophthalmologist at 1.8

Average waiting times:
General Opthalmologist: 0.29
Glaucoma Specialist: 0.00


In [11]:

def monitor(env, doctor):
    global total_doctor_wait, num_doctor_visits
    while True:
        yield env.timeout(10)  # Check every 10 time units
        if num_doctor_visits == 0:
            continue
        avg_wait_time = total_doctor_wait / num_doctor_visits
        print(f"Average doctor waiting time at {env.now} is {avg_wait_time}")
        if avg_wait_time > waiting_time_threshold:
            doctor.capacity += 1  # Add an extra doctor
            print(f"Added an extra doctor at {env.now}, new capacity: {doctor.capacity}")

def patient(env, name, doctor):
    global total_doctor_wait, num_doctor_visits

    arrival_time = env.now
    with doctor.request() as req:
        yield req
        wait_time = env.now - arrival_time
        total_doctor_wait += wait_time
        num_doctor_visits += 1
        print(f"{name} starts consultation at {env.now}, waited {wait_time}")
        yield env.timeout(random.randint(5, 10))
        print(f"{name} finishes consultation at {env.now}")

# Initialize SimPy environment
env = simpy.Environment()

# Initialize doctor resource with initial capacity of 1
doctor = simpy.Resource(env, capacity=1)

# Start the monitoring process
env.process(monitor(env, doctor))

# Generate patients
for i in range(20):
    env.process(patient(env, f"Patient-{i}", doctor))
    next_patient_in = random.randint(1, 4)
    #yield env.timeout(next_patient_in)

In [ ]:
import simpy
import numpy as np

class g:
  total_general_ophthalmologist_wait = 0
  total_glaucoma_specialist_wait = 0
  patient_count = 0
  total_patients = 30
  max_waiting_time = 4
  num_visits = 0
  max_queue_length = 10

def monitor(env, ophthalmologist):
    while True:
        yield env.timeout(2)  # Check every 2 time units
        if g.num_visits == 0:
            continue
        avg_wait_time = g.total_general_ophthalmologist_wait  / g.num_visits
        print(f"Average waiting time at {env.now} is {avg_wait_time}")
        #print(f"Number visits at {env.now} is {g.num_visits}")
        cond1 = avg_wait_time > g.max_waiting_time
        cond2 = len(ophthalmologist.queue) > g.max_queue_length
        if cond1 and cond2:
            ophthalmologist._capacity += 1  # Add an extra resource
            print(f"Added an extra ophthalmologist at {env.now}, new capacity: {ophthalmologist.capacity}")
            with ophthalmologist.request() as req:
              yield env.timeout(0)


def GlaucomaPatient(env, name, ophthalmologist):
    print(f"{name} enters the system at {env.now}")
    # Step 1: General Ophthalmologist visit
    arrival_time = env.now
    with ophthalmologist.request() as req:
        yield req
        wait_time = env.now - arrival_time
        g.num_visits += 1
        g.total_general_ophthalmologist_wait += wait_time
        print(f"{name} starts visit at general ophtalmologist at {round(env.now,2)}, waited {round(wait_time,2)}")
        print(f"Length queue at {env.now} is {len(ophthalmologist.queue)} number of visits is {g.num_visits}")
        yield env.timeout(float(np.random.normal(3, 0.1, 1)[0]))

        print(f"{name} finishes visit at general ophthalmologist at {round(env.now,2)}")


# Initialize SimPy environment
env = simpy.Environment()

# Initialize resources: general ophthalmologist, glaucoma specialist
general_ophthalmologist = simpy.Resource(env, capacity=2)

env.process(monitor(env, general_ophthalmologist))

# Generate 5 patients
for i in range(g.total_patients):
    env.process(GlaucomaPatient(env, f"Patient-{i}", general_ophthalmologist))
    ## have a time laps between patient arrivals
    env.timeout(float(np.random.normal(3, 0.1, 1)[0]))
# Run the simulation
env.run(until = 100)

# Calculate average waiting time for each resource
print("\nAverage waiting times:")
#print(f"General Opthalmologist: {g.total_general_ophthalmologist_wait / g.total_patients:.2f}")
#print(f"Glaucoma Specialist: {g.total_glaucoma_specialist_wait / g.total_patients:.2f}")

Patient-0 enters the system at 0
Patient-1 enters the system at 0
Patient-2 enters the system at 0
Patient-3 enters the system at 0
Patient-4 enters the system at 0
Patient-5 enters the system at 0
Patient-6 enters the system at 0
Patient-7 enters the system at 0
Patient-8 enters the system at 0
Patient-9 enters the system at 0
Patient-10 enters the system at 0
Patient-11 enters the system at 0
Patient-12 enters the system at 0
Patient-13 enters the system at 0
Patient-14 enters the system at 0
Patient-15 enters the system at 0
Patient-16 enters the system at 0
Patient-17 enters the system at 0
Patient-18 enters the system at 0
Patient-19 enters the system at 0
Patient-20 enters the system at 0
Patient-21 enters the system at 0
Patient-22 enters the system at 0
Patient-23 enters the system at 0
Patient-24 enters the system at 0
Patient-25 enters the system at 0
Patient-26 enters the system at 0
Patient-27 enters the system at 0
Patient-28 enters the system at 0
Patient-29 enters the sy

In [ ]:
import simpy

def monitor(env, resource, add_capacity_event):
    """Monitors the resource usage, and increases capacity if needed."""
    while True:
        yield env.timeout(1)  # Monitor every 10 time units.
        if len(resource.queue) > 2:  # Arbitrary condition to add capacity.
            resource._capacity += 1
            add_capacity_event.succeed()  # Trigger the event to notify processes.
            add_capacity_event = env.event()  # Create a new event for future capacity changes.
            print(f"Resource capacity increased to {resource.capacity} at time {env.now}")

def patient(env, name, resource, add_capacity_event):
    """Patient process that requests a resource."""
    while True:
        with resource.request() as req:
            yield req
            print(f"{name} got resource at time {env.now}")
            yield env.timeout(5)  # Use the resource for 5 time units.
            print(f"{name} released resource at time {env.now}")
            break  # Exit the loop once the resource is used.

        # Wait for the event indicating an increase in resource capacity.
        yield add_capacity_event

env = simpy.Environment()
resource = simpy.PreemptiveResource(env, capacity=1)
add_capacity_event = env.event()  # Event to trigger when capacity is increased.

# Start the monitor and patients.
env.process(monitor(env, resource, add_capacity_event))
for i in range(5):
    env.process(patient(env, f"Patient {i}", resource, add_capacity_event))

env.run(until=100)


Patient 0 got resource at time 0
Resource capacity increased to 2 at time 1
Resource capacity increased to 3 at time 2
Resource capacity increased to 4 at time 3
Resource capacity increased to 5 at time 4
Patient 0 released resource at time 5
Resource capacity increased to 6 at time 5
Patient 1 got resource at time 5
Resource capacity increased to 7 at time 6
Resource capacity increased to 8 at time 7
Resource capacity increased to 9 at time 8
Resource capacity increased to 10 at time 9
Patient 1 released resource at time 10
Resource capacity increased to 11 at time 10
Patient 2 got resource at time 10
Patient 2 released resource at time 15
Patient 3 got resource at time 15
Patient 3 released resource at time 20
Patient 4 got resource at time 20
Patient 4 released resource at time 25


A sample code for the simultaneous requests of two resources

In [ ]:
# Request a specialist visit and equipment
    with specialist.request() as specialist_req, \
            equipment.request() as equip_req:
        yield specialist_req & equip_req  # Simultaneous request for both resources

An example of inheritance

Doctor -> SpecialistDoctor

In [10]:
class Doctor:
    def __init__(self, env, capacity):
        self.resource = simpy.Resource(env, capacity)
        self.doctor_type = "General"

class SpecialistDoctor(Doctor):
    def __init__(self, env, capacity):
        super().__init__(env, capacity)
        self.doctor_type = "Specialist"